# NCBI E-utilities와 Requests를 통한 API 요청 실습

**학습목표**: Python의 `requests` 모듈을 사용해 `NCBI Entrez E-utilities`를 호출하고 소규모의 데이터 워크플로우를 직접 구축하는 방법을 배운다.

**Last updated**: 2025-11-03

**Reference**: *[A General Introduction to the E-utilities* (NCBI Bookshelf, NBK25497)](https://www.ncbi.nlm.nih.gov/books/NBK25497/).

### 핵심 개념

* **Entrez**: NCBI가 제공하는 생물의학 데이터베이스 통합 검색 시스템.
* **E-utilities**: 데이터를 요청하고 메타데이터를 반환하는 HTTP 기반 엔드포인트.
* **UID**: 각 데이터 레코드의 고유 식별자 (예: PubMed의 PMID).
* **많이 사용되는 흐름**: `ESearch` → UID 검색 → `ESummary` 또는 `EFetch` → 세부 정보 확인.
* **History Server**: `WebEnv`와 `query_key`를 사용해 결과 집합을 저장하고 재사용.

### 주요 엔드포인트

* `einfo.fcgi` — 데이터베이스 정보 확인.
* `esearch.fcgi` — 검색어로 UID 목록 조회.
* `esummary.fcgi` — UID 요약 정보 반환.
* `efetch.fcgi` — UID의 전체 레코드 반환.
* `elink.fcgi` — 데이터베이스 간 연관 링크 조회.
* `epost.fcgi` — UID 목록을 History 서버에 업로드.
* `egquery.fcgi` — 모든 데이터베이스에서의 전체 검색 결과 개수 조회.
* `espell.fcgi` — 검색어 맞춤법 제안.
* `ecitmatch.cgi` — 인용 정보를 바탕으로 PubMed ID(PMID) 조회.

### 사용 가이드라인

* **기본 URL**: `https://eutils.ncbi.nlm.nih.gov/entrez/eutils/`
* **애플리케이션 식별**: 요청 시 `tool`과 `email` 파라미터를 포함해야 함.
* **API 키(선택사항)**: 호출 한도 증가용. 환경 변수 `EUTILS_API_KEY`로 설정.
* **요청 속도 제한 준수**: 루프 내에서 짧은 대기 시간을 추가.

### 패키지 설치

```bash
pip install requests xmltodict 
```

### curl 이용해서 API 요청하고 결과 확인하기

다음 명령어를 터미널에 복사 붙여넣기 하여 실행해봅시다
```
curl -G "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi" \
  --data-urlencode "db=pubmed" \
  --data-urlencode "term=asthma treatment"
```

### Requests 라이브러리 이용하여 API 요청하고 결과 확인하기

[requests documentation](https://requests.readthedocs.io/en/latest/api/)


In [ ]:
import requests

params = {
    "db": "pubmed",
    "term": "asthma treatment",
}
r = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
                    params=params)
print("Status code:", r.status_code)

In [ ]:
?requests.get

In [ ]:
dir(r)[-20:]

In [ ]:
r.encoding

In [ ]:
r.text

In [ ]:
r.json()

In [ ]:
dict(r.headers)

In [ ]:
print("Content-Type header:", r.headers.get("content-type"))

In [ ]:
print("\n--- Raw text body (first 300 chars) ---")
print(r.text[:300])
print("\n--- Parsed JSON (if applicable) ---")
try:
    print(r.json())
except ValueError:
    print("Response is not valid JSON")

### 요청을 간편하게 만들어주는 Helper 함수 만들기

In [ ]:
# Helper functions

import os, time, urllib.parse
import requests
from dotenv import load_dotenv
load_dotenv()

BASE = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"

TOOL = os.getenv("EUTILS_TOOL", "ai-class-notebook")
EMAIL = os.getenv("EUTILS_EMAIL", "student@example.com")
API_KEY = os.getenv("EUTILS_API_KEY")  # optional

def eutils_request(endpoint, params, expect_json=True):
    """Basic request wrapper for NCBI E-utilities."""
    url = urllib.parse.urljoin(BASE, endpoint)
    params = dict(params)
    params["tool"] = TOOL
    params["email"] = EMAIL
    if API_KEY:
        params["api_key"] = API_KEY
    if expect_json:
        params.setdefault("retmode", "json")

    r = requests.get(url, params=params)
    r.raise_for_status()
    return r.json() if expect_json else r.text

In [ ]:
print("TOOL =", TOOL)
print("EMAIL =", EMAIL)
print("API_KEY set?" , bool(API_KEY))

## Requests 라이브러리 이용해서 E-utilities API 요청해보기

### EInfo: Database capabilities

#### PubMed

In [ ]:
## EInfo: Inspect a database

# Example: PubMed metadata (fields, last update, record counts)
resp = eutils_request("einfo.fcgi", {"db": "pubmed"})
print("Database:", resp["einforesult"]["dbinfo"][0]["dbname"])
print("LastUpdate:", resp["einforesult"]["dbinfo"][0]["lastupdate"])
print("\n====== LinkInfo (First 10) ======")
for link in resp["einforesult"]["dbinfo"][0]["linklist"][:10]:
    print(f'{link["name"]}: link to {link["dbto"]} DB')
print("\n====== Field Info ======")
# Show first 10 fields
fields = resp["einforesult"]["dbinfo"][0].get("fieldlist", [])
for f in fields:
    print(f'{f["name"]}: {f["fullname"]} (termcount: {f["termcount"]})')

In [ ]:
pubmed_field_len = len(resp['einforesult']['dbinfo'][0]['fieldlist'])
pubmed_link_len = len(resp['einforesult']['dbinfo'][0]['linklist'])
print(f"{pubmed_field_len=}")
print(f"{pubmed_link_len=}")

#### Clinvar

In [ ]:
## EInfo: Inspect a database

# Example: PubMed metadata (fields, last update, record counts)
resp = eutils_request("einfo.fcgi", {"db": "clinvar"})
print("Database:", resp["einforesult"]["dbinfo"][0]["dbname"])
print("LastUpdate:", resp["einforesult"]["dbinfo"][0]["lastupdate"])
print("\n====== LinkInfo (First 10)======")
for link in resp["einforesult"]["dbinfo"][0]["linklist"][:10]:
    print(f'{link["name"]}: link to {link["dbto"]} DB')
print("\n====== Field Info ======")
# Show first 10 fields
fields = resp["einforesult"]["dbinfo"][0].get("fieldlist", [])
for f in fields:
    print(f'{f["name"]}: {f["fullname"]} (termcount: {f["termcount"]})')

In [ ]:
clinvar_field_len = len(resp['einforesult']['dbinfo'][0]['fieldlist'])
clinvar_link_len = len(resp['einforesult']['dbinfo'][0]['linklist'])
print(f"{clinvar_field_len=}")
print(f"{clinvar_link_len=}")

### ESearch: PubMed를 비롯한 다양한 데이터베이스에서 검색 데이터 가져오기

Esearch는 Entrez 생태계에 있는 모든 DB에 대해 사용할 수 있습니다.  
**대표적 데이터베이스:**

* PubMed: 논문
* nucleotide: DNA / RNA 서열
* protein: 단백질 시퀀스 서열
* gene: 유전자 메타데이터
* ClinVar: 변이 정보

#### ESearch: PubMed

In [ ]:
## ESearch: Find UIDs by keyword (usehistory=y)

query = '"LLM"[Title] AND bioinformatics'
params = {
    "db": "pubmed",
    "term": query,
    "retmax": 10,
}

pubmed_resp = eutils_request("esearch.fcgi", params)
print("Query:", query)
print("Count:", pubmed_resp["esearchresult"]["count"])
pubmed_uids = pubmed_resp["esearchresult"]["idlist"]
print("First UIDs:", pubmed_uids)

In [ ]:
pubmed_resp['esearchresult']['querytranslation']

#### ESearch: PMC

In [ ]:
## ESearch: Find UIDs by keyword (usehistory=y)

query = '"LLM"[Title] AND bioinformatics'
params = {
    "db": "pmc",
    "term": query,
    "retmax": 10,
}

pmc_resp = eutils_request("esearch.fcgi", params)
print("Query:", query)
print("Count:", pmc_resp["esearchresult"]["count"])
pmc_uids = pmc_resp["esearchresult"]["idlist"]
print("First UIDs:", pmc_uids)

In [ ]:
pmc_resp['esearchresult']['querytranslation']

### EFetch: PubMed, PMC, Nucleotide 등 다양한 DB에서 특정 ID에 해당하는 상세 정보 반환

#### EFetch: Pubmed

full-text 부재, 초록 및 다양한 메타데이터 반환 가능

In [ ]:
## EFetch: Full records (XML) for one PMID

import xmltodict

if pubmed_uids:
    first_uid = pubmed_uids[0]
    xml_text = eutils_request("efetch.fcgi",
                              {"db":"pubmed", "id": first_uid, "retmode":"xml"},
                              expect_json=False)
    doc = xmltodict.parse(xml_text)
    # Extract title (best-effort)
    try:
        art = doc["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"]
        title = art.get("ArticleTitle", "")
        print("UID:", first_uid)
        print("ArticleTitle:", title)
    except Exception as e:
        print("Parsed keys:", list(doc.keys()))
else:
    print("No UIDs from ESearch step.")

In [ ]:
xml_text

In [ ]:
document_keys = list(doc.keys())
document_articleset_keys = list(doc['PubmedArticleSet'].keys())
document_article_keys = list(doc["PubmedArticleSet"]["PubmedArticle"])
document_medlinecitation_keys = list(doc["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"].keys())
article_keys = list(doc["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"].keys())
print(f"{document_keys=}")
print(f"{document_articleset_keys=}")
print(f"{document_article_keys=}")
print(f"{document_medlinecitation_keys=}")
print(f"{article_keys=}")

In [ ]:
art['Abstract']

#### EFetch: PMC

full-text 반환 가능, 등재 논문 수 적음

In [ ]:
## EFetch: Full records (XML) for one PMID

if pmc_uids:
    first_uid = pmc_uids[0]
    xml_text = eutils_request("efetch.fcgi",
                              {"db":"pmc", "id": first_uid, "retmode":"xml"},
                              expect_json=False)
    doc = xmltodict.parse(xml_text)
    # Extract title (best-effort)
    
    art = doc["pmc-articleset"]["article"]["body"]["sec"][0]
    title = art['title']
    print("UID:", first_uid)
    print("Title:", title)

else:
    print("No UIDs from ESearch step.")

In [ ]:
doc["pmc-articleset"]["article"]["body"]["sec"][3]

In [ ]:
art

In [ ]:
document_keys = list(doc.keys())
document_articleset_keys = list(doc['pmc-articleset'].keys())
document_article_keys = list(doc["pmc-articleset"]["article"])
document_medlinecitation_keys = list(doc["pmc-articleset"]["article"]["body"].keys())
article_keys = list(doc["pmc-articleset"]["article"]["body"]["sec"])
print(f"{document_keys=}")
print(f"{document_articleset_keys=}")
print(f"{document_article_keys=}")
print(f"{document_medlinecitation_keys=}")
print(f"{article_keys=}")

#### 이제 데이터 전처리만 할 수 있다면 데이터를 API를 통해 가져와서 전처리하여 내가 사용할 수 있는 데이터셋을 구축하는 것이 가능해집니다

### References

- NCBI Bookshelf — *A General Introduction to the E-utilities* (NBK25497)
- E-utilities Base: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/
- PubMed Help: https://pubmed.ncbi.nlm.nih.gov/help/
- E-utilities API parameters: https://www.ncbi.nlm.nih.gov/books/NBK25499/

### 관련 있는 Github 오픈소스 패키지

- [scholarly-python-package/scholarly](https://github.com/scholarly-python-package/scholarly)
- [jannisborn/paperscraper](https://github.com/jannisborn/paperscraper)
- [neuroquery/pubget](https://github.com/neuroquery/pubget)